In [7]:
#import necessary libraries

import glob
import numpy as np
import os
import lightgbm as lgbm
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
sns.set_theme(style="darkgrid")
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import make_scorer,mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV


In [8]:
def read_data(pth):
    df = pd.read_csv(pth,index_col='Date',parse_dates=True,na_values=['nan']).drop(['ID'],axis=1)
    df = df.fillna(method='ffill',axis=1)
    df = df.fillna(method='bfill',axis=1)
    #df.info()
    #df.describe().T
    return df

def gen_time_graph(df,stid):
    plt.figure(figsize=(18, 10))
    plt.subplot(6, 1, 1)
    sns.scatterplot(data=df, x=df.index, y='AQI',hue='AQI',size=3
    ).get_legend().remove()
    plt.ylabel('AQI')
    plt.margins(x=0)
    plt.xlabel(xlabel=None)

    plt.subplot(6, 1, 2)
    sns.scatterplot(data=df, x=df.index, y='PM2.5',hue='AQI',size=3).get_legend().remove()
    plt.ylabel('PM2.5')
    plt.margins(x=0)
    plt.xlabel(xlabel=None)

    plt.subplot(6, 1, 3)
    sns.scatterplot(data=df, x=df.index, y='PM10',hue='AQI',size=3).get_legend().remove()
    plt.ylabel('PM10')
    plt.margins(x=0)
    plt.xlabel(xlabel=None)

    plt.subplot(6, 1, 4)
    sns.scatterplot(data=df, x=df.index, y='O3',hue='AQI',size=6).get_legend().remove()
    plt.ylabel('O3')
    plt.margins(x=0)
    plt.xlabel(xlabel=None)

    plt.subplot(6, 1, 5)
    sns.scatterplot(data=df, x=df.index, y='CO',hue='AQI',size=3).get_legend().remove()
    plt.ylabel('CO')
    plt.margins(x=0)
    plt.xlabel(xlabel=None)

    plt.subplot(6, 1, 6)
    sns.scatterplot(data=df, x=df.index, y='SO2',hue='AQI',size=3).get_legend().remove()
    plt.ylabel('SO2')
    plt.margins(x=0)
    plt.xlabel(xlabel=None)

    # Set plot title
    plt.suptitle('Pollutants Over Time')

    # Adjust space between subplots
    plt.subplots_adjust(hspace=0.8)
    plt.margins(x=0)

    plt.savefig(f'./media/{stid}_Pollutans_Over_Time.jpg')
    plt.clf()

def get_heatmap(df,stid):
    corr_matrix = df[['PM2.5', 'PM10', 'O3', 'CO', 'SO2', 'AQI']].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.savefig(f'./media/{stid}_heatmap.jpg')

def get_freq_dist(df,stid):
    fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15,5))
    axes = axes.flatten()

    for i, col in enumerate(df.columns):
        ax = axes[i]
        ax.hist(df[col])
        ax.set_title(col)

    plt.suptitle('Frequency Distribution of Pollutants')
    plt.tight_layout()
    plt.savefig(f'./media/{stid}_freq_dist.jpg')

def get_monthly_trend(df, stid):
    monthly_means = df.groupby([df.index.year, df.index.month]).mean()
    monthly_means.index.rename(('year','month'),inplace=True)
    #monthly_means.head(10)
    yrs = list(monthly_means.index.get_level_values(0).unique().sort_values())
    fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15,5))
    axes = axes.flatten()
    for i, col in enumerate(monthly_means.columns):
        ax = axes[i]
        width = 0
        for yr in yrs:
            x1 = monthly_means.loc[[yr]].index.get_level_values(1).unique().sort_values()
            y1 = monthly_means.loc[[yr]][col]
            ax.bar(x1 + width, y1,width = 0.25,label=yr)
            width += 0.25        
            ax.legend()
            ax.set_title(col)
        ax.set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
        ax.set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],fontsize=9)

    plt.suptitle('Monthly Comparison of Average Value of Pollutants')
    plt.tight_layout()
    plt.savefig(f'./media/{stid}_monthy_trend.jpg')

def get_stationarity(df):
    res = ''
    for col in ['PM2.5', 'PM10', 'O3', 'CO', 'SO2']:
        adf_result = adfuller(df[col])
        if adf_result[1] > 0.05:
            res += f'{col}: time series is not stationary.\n'
        else:
            res += f'{col}: time series is stationary.\n'
    return res

def get_acf(df,stid):
    for col in ['PM2.5', 'PM10', 'O3', 'CO', 'SO2']:
        plot_acf(df[col],lags=50)
        plt.savefig(f'./media/{stid}_{col}_autocorelation.jpg')

def get_decomposition(df,stid):
    for col in ['PM2.5', 'PM10', 'O3', 'CO', 'SO2']:
        rcParams['figure.figsize'] = 15, 8
        decomposition = seasonal_decompose(df[col], model='additive',period=30)
        decomposition.plot()
        plt.savefig(f'./media/{stid}_{col}_decomposition.jpg')

def forecasting(df,daterange,stid):
    df_pol = pd.DataFrame({'ds': daterange}) 
    for col in ['PM2.5', 'PM10', 'O3', 'CO', 'SO2']:
        df_tmp = pd.DataFrame({'ds': df[col].index, 'y': df[col]})
        #df_tmp.tail()
        m = Prophet(changepoint_prior_scale= 0.5, seasonality_prior_scale= 0.01,)
        m.fit(df_tmp)
        future = pd.DataFrame({'ds':daterange})
        forecast = m.predict(future)
        forecast = forecast[['ds','yhat']].rename(columns={'yhat': col})
        df_pol = pd.merge(df_pol,forecast,on='ds',how='left')
    return df_pol

def hp_tune(df):
    param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0]
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon='28 days')
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['mae'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['mae'] = rmses
    
    best_params = all_params[np.argmin(rmses)]
    return best_params

def error_metric(y_pred,y_true):
    errors = np.abs(y_pred - y_true)
    mask = y_true > y_pred
    errors[mask] *= 1.5
    return np.mean(errors)

def LGBM_MMAE(y_pred,y_true):
    errors = np.abs(y_pred - y_true)
    mask = y_true > y_pred
    errors[mask] *= 1.5
    return ('MMAE', np.mean(errors), False)


In [ ]:
gen_grph = False
clf = 'lgbm'
files = glob.glob('./processed' + "/*.csv")

for filepath in files:
    filename = os.path.basename(filepath)
    Station_id = filename[:filename.rindex('.')]
    data = read_data(filepath)
    shape = data.shape
    row_split_no = int(np.floor(shape[0]*0.7))
    print(f'Processing {Station_id} : \n\n')
    

    start_date = data.index.max() + pd.Timedelta(days=1)
    end_date = start_date + pd.Timedelta(days=27)
    date_range = pd.date_range(start_date,end_date)

    if gen_grph:
        gen_time_graph(data,Station_id)
        get_heatmap(data,Station_id)
        get_freq_dist(data,Station_id)
        get_monthly_trend(data,Station_id)

    if gen_grph:
        result = get_stationarity(data)
        print(f'{Station_id}: \n\n {result}')

    if gen_grph:
        get_acf(data,Station_id)
        get_decomposition(data,Station_id)

    pol_pred = forecasting(data,date_range,Station_id).set_index('ds')

    X = data[['PM2.5', 'PM10', 'O3', 'CO', 'SO2']]
    Y = data['AQI']
    X_train = data.iloc[0:row_split_no][['PM2.5', 'PM10', 'O3', 'CO', 'SO2']]
    Y_train = data.iloc[0:row_split_no]['AQI']
    X_test =  data.iloc[row_split_no:][['PM2.5', 'PM10', 'O3', 'CO', 'SO2']]
    Y_test = data.iloc[row_split_no:]['AQI']

    if clf == 'lgbm':
        model = lgbm.LGBMRegressor(num_leaves=9,n_estimators=48)
        model.fit(X_train,Y_train,eval_metric=LGBM_MMAE)
        Y_pred = model.predict(X_test)

        error = mean_absolute_error(Y_test,Y_pred)
        print(f'LGBM test MAE : {error}')

        Y_pred_train = model.predict(X_train)
        error = mean_absolute_error(Y_pred_train,Y_train)
        print(f'LGBM train MAE : {error} \n')

        Y_test_forecast = model.predict(pol_pred)
        temp = pd.DataFrame({'Station_ID':Station_id + '_' + date_range.astype(str)})
        z = temp['Station_ID'].values
        predictions = np.transpose(np.vstack((z,Y_test_forecast)))

        final  = pd.DataFrame(data=predictions,columns = ['ID_Date','AQI'])
        final.to_csv('submission.csv',mode='a',index=False, header=False)

    if clf == 'ada':
        params = {'n_estimators':[1,20,30,40,50,80,100,120,150,200],'learning_rate':[0.01,0.1,1,1.5,2.0,5.0]}
        model = AdaBoostRegressor()
        grid_search = GridSearchCV(estimator=model,param_grid=params,scoring=make_scorer(error_metric,greater_is_better=False),cv=10)
        grid_search.fit(X,Y)
        print(f'\n Best params : {grid_search.best_params_}\n\n Best Score : {grid_search.best_score_}')

        best = grid_search.best_estimator_
        Y_test_forecast = best.predict(pol_pred)
        temp = pd.DataFrame({'Station_ID':Station_id + '_' + date_range.astype(str)})
        z = temp['Station_ID'].values
        predictions = np.transpose(np.vstack((z,Y_test_forecast)))

        final  = pd.DataFrame(data=predictions,columns = ['ID_Date','AQI'])
        final.to_csv('submission.csv',mode='a',index=False, header=False)
        

        